Решение интегрального уравнения первого рода
Мой вариант -- 4
Ядро = sin(2sx)

Дано ур-ие 22 (методичка стр. 23)

1) Сводим его к СЛАУ 24. Для его решения применяем метод регуляризации
Для отладки используем функции z_1(s) = 1, z_2(s) = s(1 - s), по которым строятся соответствующие правые части u(x)

2) Решаем уравнение 27 c однородными граничными условиями z(0) = z(1) = 0 (стр. 26)
Уравнение Эйлера 32, его аппроксимация приведена на стр. 27, в итоге приводим к СЛАУ, для решения которых применяем
первый способ и отлаживаем на тех же функциях

Можем регулировать поведение функций q и r
u_delta -- u с учётом погрешности машинного нуля


Необходимо сравнить результаты обоих методов, привести графики и дать практические рекомендации по решению таких задач
(выбор n и параметр регуляризации)

In [212]:
import random

import numpy as np
import pandas as pd
import plotly.express as px
from mpmath import *
from typing import Tuple

from numpy import float64, sin, cos
from scipy import integrate

In [213]:
x_label = "alpha"
y_label = "discrepancy"
color_label = "label"
columns = [x_label, y_label, color_label]
columns_without_color = [x_label, y_label]


def plot_lines(df):
    fig = px.line(df, x=x_label, y=y_label, color=color_label)
    fig.show()

def plot_lines_without_color(df):
    fig = px.line(df, x=x_label, y=y_label)
    fig.show()

In [214]:
def find_h(a: float64, b: float64, m: int) -> float64:
    return float64((b - a) / m)


def middle_rectangle_method(function, a: float64, b: float64) -> Tuple[float64, float64]:
    m = float64(pow(10, 5))
    h = float64(find_h(a, b, m))
    start_point = a + h / 2
    points = []
    sum = float64(0)

    for j in range(0, m):
        current_point = start_point + j * h
        points.append(current_point)
        sum += function(current_point)
    result = h * sum

    return result, points


def find_discrepancy(expected, actual):
    return abs(expected - actual)


def find_norm(expected, actual):
    assert expected.shape == actual.shape, f"Vectors shapes differ: {expected.shape} and {actual.shape}"
    return np.abs(1 - np.linalg.norm(actual_reg) / np.linalg.norm(expected))

In [215]:
# See task.png
# With given K(x, s) and u(x) we should find z(x)
def form_and_solve_linear_equation_for_integral_equation(kernel, u_func, alpha, n=10 ** 3):
    a = float64(0)
    b = float64(1)
    h = float64(find_h(a, b, n))
    start_point = a + h / 2
    points = []

    for j in range(n):
        points.append(start_point + j * h)

    C = np.zeros((n, n))
    U = np.zeros(n)
    for i in range(n):
        U[i] = u_func(points[i])
        for j in range(n):
            C[i, j] = h * kernel(points[i], points[j])

    solution = np.linalg.solve(C, U)
    C_conj = np.conjugate(C)
    C_reg = C_conj @ C + alpha * np.eye(n)
    U_reg = C_conj @ U
    solution_reg = np.linalg.solve(C_reg, U_reg)

    return points, solution, solution_reg

In [216]:
def form_and_solve_linear_euler_equation(kernel, kernel_1, u_func, p, r, alpha, n=10 ** 2):
    a = 0
    b = 1
    h = float64(find_h(a, b, n))
    points = np.linspace(a, b, n)

    z_0 = 0
    z_n = 0

    inside_matrices_size = n - 2
    A = np.zeros((inside_matrices_size, inside_matrices_size))
    B = np.zeros((inside_matrices_size, inside_matrices_size))
    U = np.zeros(inside_matrices_size)

    for i, point in enumerate(points[1: -1]):
        U[i] = integrate.quad(lambda t: kernel(t, point) * u_func(t), 0, 1)[0]

    for i, point_i in enumerate(points[1: -1]):
        for j, point_j in enumerate(points[1: -1]):
            B[i, j] = kernel_1(point_i, point_j)
    B = h * B

    for i, point_i in enumerate(points[1: -1]):
        if i != 0:
            A[i, i - 1] = -p / h ** 2
        A[i, i] =  2 * p / h ** 2 + r
        if i != inside_matrices_size - 1:
            A[i, i + 1] = -p / h ** 2
    A = alpha * A

    solution_reg = np.array([z_0, *np.linalg.solve(A + B, U), z_n])

    return points, solution_reg

In [217]:
np.seterr('raise')
def kernel(x, s):
    return sin(2 * x * s)

def kernel_1(s, t):
    if abs(s - t) < 10 ** (-15):
        return 1 / 2 - np.sin(4 * t) / (8 * t)
    return (np.sin(2 * (s - t)) / (s - t) - np.sin(2 * (s + t)) / (s + t)) / 4

first_test_z_function = lambda x: 1
first_test_u_function = lambda x: (sin(x) ** 2) / x

second_test_z_function = lambda x: x * (1 - x)
second_test_u_function = lambda x: (sin(x) * (sin(x) - x * cos(x))) / (2 * x ** 3)

In [225]:
data = []
for i in range(100):
    alpha = 10 ** -11
    p = random.randint(1, 10000)
    r = random.randint(1, 10000)
    points, actual_reg = form_and_solve_linear_euler_equation(kernel, kernel_1, second_test_u_function, p, r,
                                                              alpha, 1000)
    expected = np.array([second_test_z_function(point) for point in points])
    discrepancy = find_norm(expected, actual_reg)
    data.append([p, r, discrepancy])
data = sorted(data, key=lambda x: x[2])
data

[[143, 2771, 0.0006002196516977643],
 [131, 6193, 0.0007750019695950128],
 [536, 2928, 0.0007846089087851782],
 [397, 4367, 0.0007913697991427382],
 [456, 4803, 0.0008131389149497181],
 [1236, 352, 0.0008269516258735399],
 [247, 6768, 0.0008292041193995736],
 [1140, 2621, 0.0008477168520357736],
 [1631, 39, 0.0008513111224525138],
 [773, 5692, 0.0008634234618019043],
 [1523, 1868, 0.0008636423029281248],
 [2139, 410, 0.0008814390846665354],
 [1418, 4282, 0.0008831688246363489],
 [1711, 2950, 0.0008840450289615553],
 [958, 6531, 0.0008862960892703775],
 [2074, 1409, 0.0008869357125457222],
 [2257, 1802, 0.0008980397771748416],
 [2305, 2117, 0.0009025053462684562],
 [757, 8507, 0.0009043740066269068],
 [1382, 6994, 0.0009096266955149757],
 [1883, 5023, 0.0009100462143940558],
 [1654, 6086, 0.0009109450777544659],
 [2862, 1270, 0.0009174540225561634],
 [1806, 6213, 0.0009176063193349071],
 [1633, 7272, 0.0009211430859074055],
 [1645, 7842, 0.0009267441125443421],
 [3178, 1571, 0.000930744

In [224]:
data = []
for alpha_power in [i for i in range(1, -15, -1)]:
    alpha = 10 ** alpha_power
    points, actual_reg = form_and_solve_linear_euler_equation(kernel, kernel_1, second_test_u_function, 1, 1,
                                                              alpha, 1000)
    expected = np.array([second_test_z_function(point) for point in points])
    discrepancy = find_norm(expected, actual_reg)
    data.append([alpha_power, discrepancy])
    print(f"Alpha = {alpha}. Discrepancy = {discrepancy}")
df = pd.DataFrame(data, columns=columns_without_color)
plot_lines_without_color(df)

Alpha = 10. Discrepancy = 0.9999997940001777
Alpha = 1. Discrepancy = 0.9999979400057329
Alpha = 0.1. Discrepancy = 0.99997940045298
Alpha = 0.01. Discrepancy = 0.9997940440864163
Alpha = 0.001. Discrepancy = 0.997944388186086
Alpha = 0.0001. Discrepancy = 0.9798304484645766
Alpha = 1e-05. Discrepancy = 0.8302302687684935
Alpha = 1e-06. Discrepancy = 0.34269129155063116
Alpha = 1e-07. Discrepancy = 0.0775351034904389
Alpha = 1e-08. Discrepancy = 0.03543171509453891
Alpha = 1e-09. Discrepancy = 0.01618381077240061
Alpha = 1e-10. Discrepancy = 0.0033574292344241696
Alpha = 1e-11. Discrepancy = 0.0011218466994612486
Alpha = 1e-12. Discrepancy = 0.0007984241612946086
Alpha = 1e-13. Discrepancy = 0.0002568087870982172
Alpha = 1e-14. Discrepancy = 0.00040227297420347696


In [222]:
data = []
for alpha_power in [i for i in range(1, -15, -1)]:
    alpha = 10 ** alpha_power
    points, actual, actual_reg = form_and_solve_linear_equation_for_integral_equation(kernel, first_test_u_function, alpha)
    expected = np.array([first_test_z_function(point) for point in points])
    discrepancy = find_norm(expected, actual_reg)
    data.append([alpha_power, discrepancy])
    print(f"Alpha = {alpha}. Discrepancy = {discrepancy}")
df = pd.DataFrame(data, columns=columns_without_color)
plot_lines_without_color(df)

Alpha = 10. Discrepancy = 0.975617982048927
Alpha = 1. Discrepancy = 0.8040906797073082
Alpha = 0.1. Discrepancy = 0.3392591950680748
Alpha = 0.01. Discrepancy = 0.13348119316195595
Alpha = 0.001. Discrepancy = 0.0982258096454931
Alpha = 0.0001. Discrepancy = 0.07579970911324929
Alpha = 1e-05. Discrepancy = 0.06999477827123046
Alpha = 1e-06. Discrepancy = 0.06897021638411449
Alpha = 1e-07. Discrepancy = 0.06178453020842978
Alpha = 1e-08. Discrepancy = 0.05126662082621514
Alpha = 1e-09. Discrepancy = 0.04911077776619355
Alpha = 1e-10. Discrepancy = 0.048843371396130264
Alpha = 1e-11. Discrepancy = 0.047411060439621844
Alpha = 1e-12. Discrepancy = 0.040742310786149205
Alpha = 1e-13. Discrepancy = 0.037962080985637914
Alpha = 1e-14. Discrepancy = 0.03759532749390926


In [208]:
data = []
for alpha_power in [i for i in range(1, -15, -1)]:
    alpha = 10 ** alpha_power
    points, actual, actual_reg = form_and_solve_linear_equation_for_integral_equation(kernel, second_test_u_function, alpha)
    expected = np.array([second_test_z_function(point) for point in points])
    discrepancy = find_norm(expected, actual_reg)
    data.append([alpha_power, discrepancy])
    print(f"Alpha = {alpha}. Discrepancy = {discrepancy}")
df = pd.DataFrame(data, columns=columns_without_color)
plot_lines_without_color(df)

Alpha = 10. Discrepancy = 0.9768736904887867
Alpha = 1. Discrepancy = 0.8141801196491398
Alpha = 0.1. Discrepancy = 0.3732754951363907
Alpha = 0.01. Discrepancy = 0.17720770932316743
Alpha = 0.001. Discrepancy = 0.11671489636372867
Alpha = 0.0001. Discrepancy = 0.028883411842416273
Alpha = 1e-05. Discrepancy = 0.0056205964391653485
Alpha = 1e-06. Discrepancy = 0.0029104056194662054
Alpha = 1e-07. Discrepancy = 0.0019414568559712464
Alpha = 1e-08. Discrepancy = 0.0008755370323940692
Alpha = 1e-09. Discrepancy = 0.0006585656639269688
Alpha = 1e-10. Discrepancy = 0.0006334964498395657
Alpha = 1e-11. Discrepancy = 0.0005735324469977598
Alpha = 1e-12. Discrepancy = 0.00029974332227600264
Alpha = 1e-13. Discrepancy = 0.0001850208170199874
Alpha = 1e-14. Discrepancy = 0.000161058322717067
